In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import re
import math

In [2]:
df = pd.read_json("rawdatajson/products.json", lines=True)

In [3]:
df.head()

_id                                          _keywords  \
0       96085745                                                 []   
1       50097852  [anti-trace, hygiene, alcool, bille, motionsen...   
2  3560070272426  [deodorant, musk, stick, carrefour, hygiene, a...   
3  3560070937493  [lait, le, tolerance, sou, corp, dermatologiqu...   
4  3560071001308  [karite, beurre, et, de, musquee, huile, carre...   

                                           additives  \
0                                                NaN   
1   [ aqua -> fr:aqua  ]  [ aluminium-zirconium-p...   
2   [ aqua -> fr:aqua  ]  [ dipropylene-glycol ->...   
3   [ aqua -> fr:aqua  ]  [ paraffinum-liquidum -...   
4                                                NaN   

                                additives_debug_tags             additives_n  \
0                                                NaN                     NaN   
1                                                 []                       7   
2  [en-e470-fr-added, en-e490-fr-added, en-e1520-...                       3   
3  [en-e422-fr-added, en-e211-fr-added, en-e524-f...  {u'$numberLong': u'5'}   
4                                                 []                     NaN   

          additives_old_n   additives_old_tags additives_original_tags  \
0                     NaN                  NaN                     NaN   
1                       2   [en:e173, en:e460]                     NaN   
2                       2  [en:e1520, en:e110]                     NaN   
3  {u'$numberLong': u'2'}  [en:e307a, en:e330]                     NaN   
4                     NaN                   []                     NaN   

  additives_prev  additives_prev_n       ...       stores_debug_tags  \
0            NaN               NaN       ...                     NaN   
1            NaN               NaN       ...                      []   
2            NaN               NaN       ...                     NaN   
3            NaN               NaN       ...                     NaN   
4            NaN               NaN       ...                      []   

        stores_tags traces  traces_debug_tags traces_hierarchy traces_tags  \
0        [franprix]    NaN                NaN              NaN         NaN   
1        [franprix]                        []               []          []   
2  [carrefour-city]                       NaN               []          []   
3       [carrefour]                       NaN               []          []   
4       [carrefour]                        []               []          []   

  unique_scans_n unknown_nutrients_tags update_key with_sweeteners  
0            NaN                    NaN        NaN             NaN  
1            NaN                     []        NaN             NaN  
2            NaN                     []        NaN             NaN  
3            NaN                     []        NaN             NaN  
4            NaN                     []        NaN             NaN  

[5 rows x 326 columns]

In [4]:
list(df.columns)

[u'_id',
 u'_keywords',
 u'additives',
 u'additives_debug_tags',
 u'additives_n',
 u'additives_old_n',
 u'additives_old_tags',
 u'additives_original_tags',
 u'additives_prev',
 u'additives_prev_n',
 u'additives_prev_original_tags',
 u'additives_prev_tags',
 u'additives_tags',
 u'additives_tags_n',
 u'allergens',
 u'allergens_hierarchy',
 u'allergens_tags',
 u'brands',
 u'brands_debug_tags',
 u'brands_tags',
 u'categories',
 u'categories_debug_tags',
 u'categories_hierarchy',
 u'categories_prev_hierarchy',
 u'categories_prev_tags',
 u'categories_tags',
 u'checkers_tags',
 u'cities_tags',
 u'code',
 u'codes_tags',
 u'complete',
 u'completed_t',
 u'correctors_tags',
 u'countries',
 u'countries_beforescanbot',
 u'countries_debug_tags',
 u'countries_hierarchy',
 u'countries_tags',
 u'created_t',
 u'creator',
 u'debug_param_sorted_langs',
 u'debug_tags',
 u'editors',
 u'editors_tags',
 u'emb_codes',
 u'emb_codes_20141016',
 u'emb_codes_debug_tags',
 u'emb_codes_orig',
 u'emb_codes_tags',
 u'

In [5]:
df.brands.value_counts()

                                                 576
Le Petit Marseillais                              90
Carrefour                                         64
Nivea                                             63
Yves Rocher                                       54
Head & Shoulders                                  43
Schwarzkopf                                       40
Leader Price                                      37
Auchan                                            35
L'Oréal Paris,L'Oréal,Elseve                      33
Sanex                                             29
Dop                                               29
Le Petit Olivier                                  27
Dove                                              26
Diadermine                                        26
Cien                                              25
Weleda                                            23
Cora                                              23
Garnier,Ultra Doux                            

In [5]:
df.categories.value_counts()

                                                                                                                           1021
Cheveux,Shampoings                                                                                                          156
Hygiène,Gels douche,Douches et bains                                                                                        111
Cheveux,Shampoings,Shampoings anti-pelliculaires                                                                             65
Non alimentaire,Open Beauty Facts                                                                                            59
Hygiène,Savons                                                                                                               57
Hygiène,Douches et bains,Crèmes douche                                                                                       47
Hygiène,Dentifrices                                                                                     

In [6]:
df.shape 

(4346, 326)

In [7]:
df[1500:1520]

_id                                          _keywords  \
1500  0738678240472  [ultra, gliding, gel, rasage, huile, shave, cr...   
1501  0738678239940                               [shaving, gel, crew]   
1502       20553531                                             [cien]   
1503          06582  [san, composant, du, bille, packaging, pa, deo...   
1504  3538394920033  [cosly, normaux, pre, cosmetique, la, shampoin...   
1505  3538394612938  [cosly, douche, savon, san, ultra, doux, fruit...   
1506  3538394612839  [cosly, savon, san, synthese, ulta, aux, cosme...   
1507  3538394612730  [cosly, douche, savon, san, ultra, doux, aux, ...   
1508  3532281624608  [comestible, quality, fraise, de, pour, ph, ge...   
1509  3532281625605  [san, compatible, ph, le, neutre, body, la, ge...   
1510       20297190  [hydratante, le, creme, dermatologiquement, po...   
1511  5011417559161  [remove, wash, of, 99-9, kill, dettol, viruse,...   
1512  5011417559185  [remove, wash, tested, of, 99-9, kill, nourish...   
1513  5011417560853                                                 []   
1514  5011522037059  [eylure, faux, le, maquillage, lengthening, po...   
1515  5012874022441  [the, stick, le, maquillage, concealer, fond, ...   
1516  5011321489851  [gamble, dentaire, ufsbd, creme, procter, appa...   
1517  5011321260696  [duschgele, old, and, bath, spice, shower, hyg...   
1518  5011321532939  [antipelliculaire, shoulder, cheveux, apres-sh...   
1519  5011321433991  [antipelliculaire, shoulder, cheveux, classic,...   

                                              additives  \
1500   [ olive-oil-peg-7-esters -> fr:olive-oil-peg-...   
1501   [ aqua -> en:aqua  ]  [ dicaprylyl-carbonate ...   
1502                                                NaN   
1503   [ aqua -> fr:aqua  ]  [ aluminum-chlorohydrat...   
1504   [ aqua -> fr:aqua  ]  [ eau -> fr:eau  ]  [ s...   
1505   [ aqua -> fr:aqua  ]  [ water -> fr:water  ] ...   
1506   [ aqua -> fr:aqua  ]  [ water -> fr:water  ] ...   
1507   [ aqua -> fr:aqua  ]  [ water -> fr:water  ] ...   
1508   [ aqua -> fr:aqua  ]  [ glycerin -> en:e422  ...   
1509   [ glycerin -> en:e422  -> exists  ]  [ polyso...   
1510   [ aqua -> fr:aqua  ]  [ glycerin -> en:e422  ...   
1511                                                NaN   
1512                                                NaN   
1513                                                NaN   
1514                                                NaN   
1515                                                NaN   
1516   [ calcium-zinc-pvm-ma-copolymer -> fr:calcium...   
1517   [ aqua -> fr:aqua  ]  [ sodium-laureth-sulfat...   
1518   [ aqua -> fr:aqua  ]  [ stearyl-alcohol -> fr...   
1519   [ aqua -> fr:aqua  ]  [ stearyl-alcohol -> fr...   

                                   additives_debug_tags  \
1500                                                 []   
1501  [en-e502-other-added, en-e422-other-added, en-...   
1502                                                 []   
1503  [en-e173-fr-added, en-e211-fr-added, en-e490-f...   
1504  [en-e470-fr-added, en-e508-fr-added, en-e422-f...   
1505  [en-e470-fr-added, en-e422-fr-added, en-e508-f...   
1506  [en-e470-fr-added, en-e422-fr-added, en-e508-f...   
1507  [en-e470-fr-added, en-e508-fr-added, en-e422-f...   
1508                                                 []   
1509               [en-e422-fr-added, en-e270-fr-added]   
1510  [en-e422-fr-added, en-e1440-fr-added, en-e218-...   
1511                                                 []   
1512                                                 []   
1513                                                 []   
1514                                                 []   
1515                                                 []   
1516  [en-e341-fr-added, en-e466-fr-added, en-e122-f...   
1517  [en-e470-fr-added, en-e508-fr-added, en-e487-f...   
1518  [en-e620-fr-added, en-e1519-fr-added, en-e508-...   
1519  [en-e620-fr-added, en-e1519-fr-adde

In [8]:
allergens = [x for x in list(df.allergens_tags.values) if type(x) == list and len(x)>0]


In [9]:
allergens

[[u'macadamia'],
 [u'prunus-amygdalus'],
 [u'prunus-amygdalus'],
 [u'prunus-amygdalus', u'', u'', u'almond'],
 [u'prunus-amygdalus'],
 [u'prunus-amygdalus', u'', u'', u'prunus-amygdalus'],
 [u'prunus-amygdalus'],
 [u'sweet-almond'],
 [u'prunus-amygdalus'],
 [u'prunus-amygdalus'],
 [u'almond'],
 [u'almond'],
 [u'prunus-amygdalus'],
 [u'prunus-amygdalus'],
 [u'prunus-amygdalus', u'', u'', u'almond'],
 [u'prunus-amygdalus', u'', u'', u'almond'],
 [u'macadamia'],
 [u'macadamia'],
 [u'prunus-amygdalus', u'', u'', u'almond'],
 [u'prunus-amygdalus', u'', u'', u'almond'],
 [u'aluminum', u'', u'', u'cinnamal', u'', u'', u'citral'],
 [u'sodium-monofluorophosphate'],
 [u'prunus-amygdalus'],
 [u'prunus-amygdalus'],
 [u'prunus-amygdalus', u'', u'', u'almond'],
 [u'prunus-amygdalus', u'', u'', u'almond'],
 [u'prunus-amygdalus'],
 [u'prunus-amygdalus'],
 [u'prunus-amygdalus'],
 [u'prunus-amygdalus', u'', u'', u'macadamia'],
 [u'prunus-amygdalus'],
 [u'prunus-amygdalus'],
 [u'prunus-amygdalus'],
 [u'p

In [10]:
### what are these allergens?
### how many allergens are there?
len(allergens)

58

In [11]:
####
## prunus-amygdalus is almond, a tree nut
## aluminum is aluminum
## cinnamal is the naturally occuring chemical in cinnamon that gives it its flavor, banned for cosmetic use in EU
## citral is a chemical commonly found in lemon scented plants that is known to be a skin irritant
## macadamia is macademia nut, a tree nut
## arachidyl is derived from peanuts
## d-limonene is a constitutent of citurs and is known to cause skin irritation in some individuals
## linalool is a naturally occuring scent in lavendar, mint, orange blossoms, cardamom and others.  It causes skin allergies in about 5% of the Swedish population
## geraniol is naturally occuring and found in many essential oils that smell like roses.  It is banned in EU as a skin irritant
## alpha-isomethyl-ionone is an ingredient in bubble bath and aftershave that can cause skin irritation
## coumarin gives many plants their sweet smell and is a known human irritant.  Banned in EU
## citronellol is found in roses and geraniums.  It is restricted in perfume

### Sources https://www.ewg.org/skindeep/ , http://en.wikipedia.org

In [12]:
main_allergens = [x for x in list(df.allergens.values) if type(x) == str and len(x) >0]

In [13]:
type(df.allergens[75])
#type(df.allergens.values[20])

unicode

In [14]:
main_allergens

[]

In [15]:
len(main_allergens)

0

In [16]:
cats = [x for x in df['categories_hierarchy'] if type(x) == list  and len(x) >0]

In [17]:
type(df['categories_hierarchy'][0])

float

In [18]:
len(cats)

2619

In [19]:
### half of all cosmetics have category hierarchies

In [20]:
cats_general = [x for x in df['categories'] if type(x) == str and len(x)>0]

In [21]:
len(cats_general)

0

In [22]:
cats_general

[]

In [23]:
cat_tags = [x for x in df['categories_tags'] if type(x) == list  and len(x) >0]

In [24]:
cat_tags

[[u'en:hygiene',
  u'en:anti-perspirants',
  u'en:deodorants',
  u'en:roll-on-deodorants'],
 [u'en:hygiene', u'en:deodorants', u'fr:deodorants-sticks'],
 [u'en:body', u'en:body-milks'],
 [u'en:hygiene', u'en:hair-removal'],
 [u'en:hygiene', u'en:intimate-hygiene', u'fr:gels-pour-la-toilette-intime'],
 [u'en:body', u'en:body-creams', u'fr:cremes-pour-les-pieds'],
 [u'en:body', u'en:body-creams', u'fr:cremes-pour-les-pieds'],
 [u'en:hygiene', u'en:soaps', u'fr:savons-liquides'],
 [u'en:hygiene', u'en:soaps', u'fr:savons-liquides'],
 [u'en:shaving', u'en:aftershaves', u'fr:baumes-apres-rasage'],
 [u'en:hair', u'en:shampoos', u'en:anti-dandruff-shampoos'],
 [u'en:hair', u'en:shampoos', u'en:anti-dandruff-shampoos'],
 [u'en:hair', u'en:hair-care'],
 [u'en:hygiene', u'en:soaps', u'fr:savons-liquides'],
 [u'en:hygiene', u'en:soaps', u'fr:savons-liquides'],
 [u'en:hair',
  u'en:hygiene',
  u'en:shampoos',
  u'en:2-in-1-shampoos',
  u'en:shampoos-shower-gels',
  u'en:shower-gels',
  u'en:shower

In [25]:
len(cat_tags)

2619

In [26]:
for x in df.columns:
    print (x, type(df[x][0]))


(u'_id', <type 'unicode'>)
(u'_keywords', <type 'list'>)
(u'additives', <type 'float'>)
(u'additives_debug_tags', <type 'float'>)
(u'additives_n', <type 'float'>)
(u'additives_old_n', <type 'float'>)
(u'additives_old_tags', <type 'float'>)
(u'additives_original_tags', <type 'float'>)
(u'additives_prev', <type 'float'>)
(u'additives_prev_n', <type 'numpy.float64'>)
(u'additives_prev_original_tags', <type 'float'>)
(u'additives_prev_tags', <type 'float'>)
(u'additives_tags', <type 'float'>)
(u'additives_tags_n', <type 'numpy.float64'>)
(u'allergens', <type 'float'>)
(u'allergens_hierarchy', <type 'float'>)
(u'allergens_tags', <type 'float'>)
(u'brands', <type 'float'>)
(u'brands_debug_tags', <type 'float'>)
(u'brands_tags', <type 'float'>)
(u'categories', <type 'float'>)
(u'categories_debug_tags', <type 'float'>)
(u'categories_hierarchy', <type 'float'>)
(u'categories_prev_hierarchy', <type 'float'>)
(u'categories_prev_tags', <type 'float'>)
(u'categories_tags', <type 'float'>)
(u'checke

In [27]:
### drop rows that are not relevant.  Define relevant as if categories_tags is filled
idx = []
for i, val in enumerate(df['categories_tags']):
    #print i, val, type(val)
    if (type(val) != list) or (type(val) == list and len(val)  == 0 ):
        idx.append(i)
idx[:7]

[0, 5, 20, 39, 55, 58, 60]

In [28]:
df1 = df.drop(df.index[idx])
df1.head(10)

_id                                          _keywords  \
1        50097852  [anti-trace, hygiene, alcool, bille, motionsen...   
2   3560070272426  [deodorant, musk, stick, carrefour, hygiene, a...   
3   3560070937493  [lait, le, tolerance, sou, corp, dermatologiqu...   
4   3560071001308  [karite, beurre, et, de, musquee, huile, carre...   
6   3560070416974  [la, intime, carrefour, bio, gel, ecocert, pou...   
7   3560070231584  [le, gommant, corne, pied, creme, carrefour, p...   
8   3560070231614  [le, nourrissant, pied, creme, carrefour, pour...   
9   3560070923281  [savon, mousant, carrefour, gel, dermatologiqu...   
10  3560070923250                        [liquide, savon, carrefour]   
11  3560070922789  [le, cosmetique, activ, apres-rasage, rasage, ...   

                                            additives  \
1    [ aqua -> fr:aqua  ]  [ aluminium-zirconium-p...   
2    [ aqua -> fr:aqua  ]  [ dipropylene-glycol ->...   
3    [ aqua -> fr:aqua  ]  [ paraffinum-liquidum -...   
4                                                 NaN   
6    [ aqua -> fr:aqua  ]  [ citrus-aurantium-amar...   
7    [ aqua -> fr:aqua  ]  [ paraffin -> fr:paraff...   
8    [ aqua -> fr:aqua  ]  [ vitis-vinifera-seed-o...   
9    [ aqua -> fr:aqua  ]  [ sodium-laureth-sulfat...   
10   [ aqua -> fr:aqua  ]  [ sodium-laureth-sulfat...   
11   [ aqua -> fr:aqua  ]  [ cyclopentasiloxane ->...   

                                 additives_debug_tags  \
1                                                  []   
2   [en-e470-fr-added, en-e490-fr-added, en-e1520-...   
3   [en-e422-fr-added, en-e211-fr-added, en-e524-f...   
4                                                  []   
6   [en-e422-fr-added, en-e470-fr-added, en-e1519-...   
7                                  [en-e218-fr-added]   
8   [en-e927b-fr-added, en-e570-fr-added, en-e490-...   
9   [en-e470-fr-added, en-e508-fr-added, en-e422-f...   
10                                                 []   
11  [en-e524-fr-added, en-e490-fr-added, en-e1519-...   

                additives_n         additives_old_n  \
1                         7                       2   
2                         3                       2   
3    {u'$numberLong': u'5'}  {u'$numberLong': u'2'}   
4                       NaN                     NaN   
6   {u'$numberLong': u'11'}  {u'$numberLong': u'3'}   
7    {u'$numberLong': u'6'}  {u'$numberLong': u'5'}   
8   {u'$numberLong': u'13'}  {u'$numberLong': u'5'}   
9   {u'$numberLong': u'11'}  {u'$numberLong': u'3'}   
10                       10                       3   
11   {u'$numberLong': u'4'}  {u'$numberLong': u'1'}   

                                 additives_old_tags  \
1                                [en:e173, en:e460]   
2                               [en:e1520, en:e110]   
3                               [en:e307a, en:e330]   
4                                                []   
6                      [en:e415, en:e330, en:e307a]   
7    [en:e900, en:e415, en:e330, en:e307a, en:e321]   
8   [en:e1520, en:e900, en:e330, en:e307a, en:e321]   
9                      [en:e330, en:e307a, en:e110]   
10                      [en:e330, en:e133, en:e151]   
11                                       [en:e1520]   

                              additives_original_tags additives_prev  \
1                                                 NaN            NaN   
2                                                 NaN            NaN   
3                                                 NaN            NaN   
4                                                 NaN            NaN   
6                                                 NaN            NaN   
7                                                 NaN            NaN   
8                                                 NaN            NaN   
9                                                 NaN            NaN   
10  [en:e470, en:e508, en:e422, en:e1519, en:e330,...            NaN   
11                               

In [29]:
df1.shape

(2619, 326)

In [30]:
# re-index
df2 = df1.reset_index(drop=True)
df2.head(10)

_id                                          _keywords  \
0       50097852  [anti-trace, hygiene, alcool, bille, motionsen...   
1  3560070272426  [deodorant, musk, stick, carrefour, hygiene, a...   
2  3560070937493  [lait, le, tolerance, sou, corp, dermatologiqu...   
3  3560071001308  [karite, beurre, et, de, musquee, huile, carre...   
4  3560070416974  [la, intime, carrefour, bio, gel, ecocert, pou...   
5  3560070231584  [le, gommant, corne, pied, creme, carrefour, p...   
6  3560070231614  [le, nourrissant, pied, creme, carrefour, pour...   
7  3560070923281  [savon, mousant, carrefour, gel, dermatologiqu...   
8  3560070923250                        [liquide, savon, carrefour]   
9  3560070922789  [le, cosmetique, activ, apres-rasage, rasage, ...   

                                           additives  \
0   [ aqua -> fr:aqua  ]  [ aluminium-zirconium-p...   
1   [ aqua -> fr:aqua  ]  [ dipropylene-glycol ->...   
2   [ aqua -> fr:aqua  ]  [ paraffinum-liquidum -...   
3                                                NaN   
4   [ aqua -> fr:aqua  ]  [ citrus-aurantium-amar...   
5   [ aqua -> fr:aqua  ]  [ paraffin -> fr:paraff...   
6   [ aqua -> fr:aqua  ]  [ vitis-vinifera-seed-o...   
7   [ aqua -> fr:aqua  ]  [ sodium-laureth-sulfat...   
8   [ aqua -> fr:aqua  ]  [ sodium-laureth-sulfat...   
9   [ aqua -> fr:aqua  ]  [ cyclopentasiloxane ->...   

                                additives_debug_tags              additives_n  \
0                                                 []                        7   
1  [en-e470-fr-added, en-e490-fr-added, en-e1520-...                        3   
2  [en-e422-fr-added, en-e211-fr-added, en-e524-f...   {u'$numberLong': u'5'}   
3                                                 []                      NaN   
4  [en-e422-fr-added, en-e470-fr-added, en-e1519-...  {u'$numberLong': u'11'}   
5                                 [en-e218-fr-added]   {u'$numberLong': u'6'}   
6  [en-e927b-fr-added, en-e570-fr-added, en-e490-...  {u'$numberLong': u'13'}   
7  [en-e470-fr-added, en-e508-fr-added, en-e422-f...  {u'$numberLong': u'11'}   
8                                                 []                       10   
9  [en-e524-fr-added, en-e490-fr-added, en-e1519-...   {u'$numberLong': u'4'}   

          additives_old_n                               additives_old_tags  \
0                       2                               [en:e173, en:e460]   
1                       2                              [en:e1520, en:e110]   
2  {u'$numberLong': u'2'}                              [en:e307a, en:e330]   
3                     NaN                                               []   
4  {u'$numberLong': u'3'}                     [en:e415, en:e330, en:e307a]   
5  {u'$numberLong': u'5'}   [en:e900, en:e415, en:e330, en:e307a, en:e321]   
6  {u'$numberLong': u'5'}  [en:e1520, en:e900, en:e330, en:e307a, en:e321]   
7  {u'$numberLong': u'3'}                     [en:e330, en:e307a, en:e110]   
8                       3                      [en:e330, en:e133, en:e151]   
9  {u'$numberLong': u'1'}                                       [en:e1520]   

                             additives_original_tags additives_prev  \
0                                                NaN            NaN   
1                                                NaN            NaN   
2                                                NaN            NaN   
3                                                NaN            NaN   
4                                                NaN            NaN   
5                                                NaN            NaN   
6                                                NaN            NaN   
7                                                NaN            NaN   
8  [en:e470, en:e508, en:e422, en:e1519, en:e330,...            NaN   
9                                                NaN            NaN   

   additives_prev_n       ...       stores_debug_tags       stores_tags  \
0     

In [31]:
### check for completeness
cat_completeness = []
types = set()

for x in df2.columns:
    t = type(df2[x][1])
    types.add(t)
    df2[x] = df2[x].apply(lambda y: np.nan if (type(y)==list or type(y)==str) and len(y)==0 else y)
    df2[x] = df2[x].apply(lambda y: np.nan if type(y)==dict and (not bool(y)) else y)
    ln = df2[x].notnull().sum()
    threshold = ['Over Threshold' if ln > 2619*0.95 else 'Under Threshold'][0]
    cat_completeness.append({'cat_name': x, 
                            'cat_type': t,
                            'cat_valid_count': ln,
                            'cat_completeness': threshold})

In [32]:
cat_completeness[:20]

[{'cat_completeness': 'Over Threshold',
  'cat_name': u'_id',
  'cat_type': unicode,
  'cat_valid_count': 2619},
 {'cat_completeness': 'Over Threshold',
  'cat_name': u'_keywords',
  'cat_type': list,
  'cat_valid_count': 2619},
 {'cat_completeness': 'Under Threshold',
  'cat_name': u'additives',
  'cat_type': unicode,
  'cat_valid_count': 2170},
 {'cat_completeness': 'Under Threshold',
  'cat_name': u'additives_debug_tags',
  'cat_type': list,
  'cat_valid_count': 1551},
 {'cat_completeness': 'Under Threshold',
  'cat_name': u'additives_n',
  'cat_type': unicode,
  'cat_valid_count': 2172},
 {'cat_completeness': 'Under Threshold',
  'cat_name': u'additives_old_n',
  'cat_type': unicode,
  'cat_valid_count': 2172},
 {'cat_completeness': 'Under Threshold',
  'cat_name': u'additives_old_tags',
  'cat_type': list,
  'cat_valid_count': 1881},
 {'cat_completeness': 'Under Threshold',
  'cat_name': u'additives_original_tags',
  'cat_type': float,
  'cat_valid_count': 268},
 {'cat_completenes

In [33]:
### how many were not complete?
cols2 = [x['cat_name'] for x in cat_completeness if x['cat_completeness'] == 'Over Threshold']
len(cols2)

60

In [34]:
### create a df that only contains the completed columns
df3 = df2[cols2]

In [35]:
df3.head()

_id                                          _keywords allergens  \
0       50097852  [anti-trace, hygiene, alcool, bille, motionsen...             
1  3560070272426  [deodorant, musk, stick, carrefour, hygiene, a...             
2  3560070937493  [lait, le, tolerance, sou, corp, dermatologiqu...             
3  3560071001308  [karite, beurre, et, de, musquee, huile, carre...             
4  3560070416974  [la, intime, carrefour, bio, gel, ecocert, pou...             

                    brands                 brands_tags  \
0          Rexona,Unilever          [rexona, unilever]   
1                Carrefour                 [carrefour]   
2                Carrefour                 [carrefour]   
3                Carrefour                 [carrefour]   
4  Carrefour Bio,Carrefour  [carrefour-bio, carrefour]   

                                          categories  \
0  Hygiène,Déodorants anti-transpirants,Déodorant...   
1               Hygiène,Déodorants,Déodorants sticks   
2                                Laits pour le corps   
3                                          Épilation   
4  Hygiène,Hygiène intime,Gels pour la toilette i...   

                                categories_hierarchy  \
0  [en:hygiene, en:anti-perspirants, en:deodorant...   
1  [en:hygiene, en:deodorants, fr:deodorants-sticks]   
2                           [en:body, en:body-milks]   
3                      [en:hygiene, en:hair-removal]   
4  [en:hygiene, en:intimate-hygiene, fr:gels-pour...   

                                     categories_tags           code  \
0  [en:hygiene, en:anti-perspirants, en:deodorant...       50097852   
1  [en:hygiene, en:deodorants, fr:deodorants-sticks]  3560070272426   
2                           [en:body, en:body-milks]  3560070937493   
3                      [en:hygiene, en:hair-removal]  3560071001308   
4  [en:hygiene, en:intimate-hygiene, fr:gels-pour...  3560070416974   

                                          codes_tags  ...   quantity  \
0  [code-8, 50097852, 5009785x, 500978xx, 50097xx...  ...      50 ml   
1  [code-13, 3560070272426, 356007027242x, 356007...  ...      50 ml   
2  [code-13, 3560070937493, 356007093749x, 356007...  ...     400 ml   
3  [code-13, 3560071001308, 356007100130x, 356007...  ...     100 ml   
4  [code-13, 3560070416974, 356007041697x, 356007...  ...     200 ml   

                       rev        serving_quantity serving_size  \
0                       14                       0                
1                       13                       0                
2  {u'$numberLong': u'11'}  {u'$numberLong': u'0'}                
3                        8                       0                
4  {u'$numberLong': u'15'}  {u'$numberLong': u'0'}                

                           sortkey  \
0                       1487927561   
1                       1491076400   
2  {u'$numberLong': u'1459971277'}   
3                        487925728   
4  {u'$numberLong': u'1463774184'}   

                                              states  \
0  en:to-be-checked, en:complete, en:nutrition-fa...   
1  en:to-be-checked, en:complete, en:nutrition-fa...   
2  en:to-be-checked, en:complete, en:nutrition-fa...   
3  en:to-be-completed, en:nutrition-facts-complet...   
4  en:to-be-checked, en:complete, en:nutrition-fa...   

                                    states_hierarchy  \
0  [en:to-be-checked, en:complete, en:nutrition-f...   
1  [en:to-be-checked, en:complete, en:nutrition-f...   
2  [en:to-be-checked, en:complete, en:nutrition-f...   
3  [en:to-be-completed, en:nutrition-facts-comple...   
4  [en:to-be-checked, en:complete, en:nutrition-f...   

                                         states_tags          stores traces  
0  [en:to-be-checked, en:complete, en:nutrition-f...        Franprix         
1  [en:to-be-checked, en:complete, en:nutrition-f...  Carrefour City         
2  [en:to-be-checked, en:complete, en:nutrition-f...       Carrefour         
3  [en:to-be-completed, e

In [45]:
df3.shape

(2619, 60)

In [46]:
df3.columns

Index([u'_id', u'_keywords', u'allergens', u'brands', u'brands_tags',
       u'categories', u'categories_hierarchy', u'categories_tags', u'code',
       u'codes_tags', u'complete', u'correctors_tags', u'countries',
       u'countries_hierarchy', u'countries_tags', u'created_t', u'creator',
       u'editors_tags', u'emb_codes', u'emb_codes_orig', u'entry_dates_tags',
       u'expiration_date', u'generic_name', u'id', u'informers_tags',
       u'ingredients_text', u'ingredients_text_debug',
       u'ingredients_text_with_allergens', u'interface_version_created',
       u'interface_version_modified', u'labels', u'lang', u'languages',
       u'languages_codes', u'languages_hierarchy', u'languages_tags',
       u'last_edit_dates_tags', u'last_modified_by', u'last_modified_t', u'lc',
       u'link', u'manufacturing_places', u'no_nutrition_data',
       u'nutrition_data_per', u'nutrition_grades_tags',
       u'nutrition_score_debug', u'origins', u'packaging', u'product_name',
       u'purchas

In [37]:
len(df3)

2619

In [38]:
df4 = df3.dropna()
df4.shape
df4.head(10)
d4['ingredients'] = df3['ingredients']

_id                                          _keywords allergens  \
0       50097852  [anti-trace, hygiene, alcool, bille, motionsen...             
1  3560070272426  [deodorant, musk, stick, carrefour, hygiene, a...             
2  3560070937493  [lait, le, tolerance, sou, corp, dermatologiqu...             
3  3560071001308  [karite, beurre, et, de, musquee, huile, carre...             
4  3560070416974  [la, intime, carrefour, bio, gel, ecocert, pou...             
5  3560070231584  [le, gommant, corne, pied, creme, carrefour, p...             
6  3560070231614  [le, nourrissant, pied, creme, carrefour, pour...             
7  3560070923281  [savon, mousant, carrefour, gel, dermatologiqu...             
8  3560070923250                        [liquide, savon, carrefour]             
9  3560070922789  [le, cosmetique, activ, apres-rasage, rasage, ...             

                                              brands  \
0                                    Rexona,Unilever   
1                                          Carrefour   
2                                          Carrefour   
3                                          Carrefour   
4                            Carrefour Bio,Carrefour   
5                                          Carrefour   
6                                          Carrefour   
7                                          Carrefour   
8                                          Carrefour   
9  Men'Activ Science,Men'Activ,Les Cosmétiques De...   

                                         brands_tags  \
0                                 [rexona, unilever]   
1                                        [carrefour]   
2                                        [carrefour]   
3                                        [carrefour]   
4                         [carrefour-bio, carrefour]   
5                                        [carrefour]   
6                                        [carrefour]   
7                                        [carrefour]   
8                                        [carrefour]   
9  [men-activ-science, men-activ, les-cosmetiques...   

                                          categories  \
0  Hygiène,Déodorants anti-transpirants,Déodorant...   
1               Hygiène,Déodorants,Déodorants sticks   
2                                Laits pour le corps   
3                                          Épilation   
4  Hygiène,Hygiène intime,Gels pour la toilette i...   
5                              Crèmes pour les pieds   
6                              Crèmes pour les pieds   
7                     Hygiène,Savons,Savons liquides   
8                                    Savons liquides   
9                                Baumes après-rasage   

                                categories_hierarchy  \
0  [en:hygiene, en:anti-perspirants, en:deodorant...   
1  [en:hygiene, en:deodorants, fr:deodorants-sticks]   
2                           [en:body, en:body-milks]   
3                      [en:hygiene, en:hair-removal]   
4  [en:hygiene, en:intimate-hygiene, fr:gels-pour...   
5  [en:body, en:body-creams, fr:cremes-pour-les-p...   
6  [en:body, en:body-creams, fr:cremes-pour-les-p...   
7         [en:hygiene, en:soaps, fr:savons-liquides]   
8         [en:hygiene, en:soaps, fr:savons-liquides]   
9  [en:shaving, en:aftershaves, fr:baumes-apres-r...   

                                     categories_tags           code  \
0  [en:hygiene, en:anti-perspirants, en:deodorant...       50097852   
1  [en:hygiene, en:deodorants, fr:deodorants-sticks]  3560070272426   
2                           [en:body, en:body-milks]  3560070937493   
3                      [en:hygiene, en:hair-removal]  3560071001308   
4  [en:hygiene, en:intimate-hygiene, fr:gels-pour...  3560070416974   
5  [en:body, en:body-creams, fr:cremes-pour-les-p...  3560070231584   
6  [en:body, en:body-creams, fr:cremes-pour-les-p...  3560070231614   
7         [en:hygiene, en:soaps, fr:savons-liquides]  3560070923281   
8         [en:

In [47]:
df4.shape

(2293, 60)

In [48]:
df4.columns

Index([u'_id', u'_keywords', u'allergens', u'brands', u'brands_tags',
       u'categories', u'categories_hierarchy', u'categories_tags', u'code',
       u'codes_tags', u'complete', u'correctors_tags', u'countries',
       u'countries_hierarchy', u'countries_tags', u'created_t', u'creator',
       u'editors_tags', u'emb_codes', u'emb_codes_orig', u'entry_dates_tags',
       u'expiration_date', u'generic_name', u'id', u'informers_tags',
       u'ingredients_text', u'ingredients_text_debug',
       u'ingredients_text_with_allergens', u'interface_version_created',
       u'interface_version_modified', u'labels', u'lang', u'languages',
       u'languages_codes', u'languages_hierarchy', u'languages_tags',
       u'last_edit_dates_tags', u'last_modified_by', u'last_modified_t', u'lc',
       u'link', u'manufacturing_places', u'no_nutrition_data',
       u'nutrition_data_per', u'nutrition_grades_tags',
       u'nutrition_score_debug', u'origins', u'packaging', u'product_name',
       u'purchas

In [50]:
df4.head()

_id                                          _keywords allergens  \
0       50097852  [anti-trace, hygiene, alcool, bille, motionsen...             
1  3560070272426  [deodorant, musk, stick, carrefour, hygiene, a...             
2  3560070937493  [lait, le, tolerance, sou, corp, dermatologiqu...             
3  3560071001308  [karite, beurre, et, de, musquee, huile, carre...             
4  3560070416974  [la, intime, carrefour, bio, gel, ecocert, pou...             

                    brands                 brands_tags  \
0          Rexona,Unilever          [rexona, unilever]   
1                Carrefour                 [carrefour]   
2                Carrefour                 [carrefour]   
3                Carrefour                 [carrefour]   
4  Carrefour Bio,Carrefour  [carrefour-bio, carrefour]   

                                          categories  \
0  Hygiène,Déodorants anti-transpirants,Déodorant...   
1               Hygiène,Déodorants,Déodorants sticks   
2                                Laits pour le corps   
3                                          Épilation   
4  Hygiène,Hygiène intime,Gels pour la toilette i...   

                                categories_hierarchy  \
0  [en:hygiene, en:anti-perspirants, en:deodorant...   
1  [en:hygiene, en:deodorants, fr:deodorants-sticks]   
2                           [en:body, en:body-milks]   
3                      [en:hygiene, en:hair-removal]   
4  [en:hygiene, en:intimate-hygiene, fr:gels-pour...   

                                     categories_tags           code  \
0  [en:hygiene, en:anti-perspirants, en:deodorant...       50097852   
1  [en:hygiene, en:deodorants, fr:deodorants-sticks]  3560070272426   
2                           [en:body, en:body-milks]  3560070937493   
3                      [en:hygiene, en:hair-removal]  3560071001308   
4  [en:hygiene, en:intimate-hygiene, fr:gels-pour...  3560070416974   

                                          codes_tags  ...   quantity  \
0  [code-8, 50097852, 5009785x, 500978xx, 50097xx...  ...      50 ml   
1  [code-13, 3560070272426, 356007027242x, 356007...  ...      50 ml   
2  [code-13, 3560070937493, 356007093749x, 356007...  ...     400 ml   
3  [code-13, 3560071001308, 356007100130x, 356007...  ...     100 ml   
4  [code-13, 3560070416974, 356007041697x, 356007...  ...     200 ml   

                       rev        serving_quantity serving_size  \
0                       14                       0                
1                       13                       0                
2  {u'$numberLong': u'11'}  {u'$numberLong': u'0'}                
3                        8                       0                
4  {u'$numberLong': u'15'}  {u'$numberLong': u'0'}                

                           sortkey  \
0                       1487927561   
1                       1491076400   
2  {u'$numberLong': u'1459971277'}   
3                        487925728   
4  {u'$numberLong': u'1463774184'}   

                                              states  \
0  en:to-be-checked, en:complete, en:nutrition-fa...   
1  en:to-be-checked, en:complete, en:nutrition-fa...   
2  en:to-be-checked, en:complete, en:nutrition-fa...   
3  en:to-be-completed, en:nutrition-facts-complet...   
4  en:to-be-checked, en:complete, en:nutrition-fa...   

                                    states_hierarchy  \
0  [en:to-be-checked, en:complete, en:nutrition-f...   
1  [en:to-be-checked, en:complete, en:nutrition-f...   
2  [en:to-be-checked, en:complete, en:nutrition-f...   
3  [en:to-be-completed, en:nutrition-facts-comple...   
4  [en:to-be-checked, en:complete, en:nutrition-f...   

                                         states_tags          stores traces  
0  [en:to-be-checked, en:complete, en:nutrition-f...        Franprix         
1  [en:to-be-checked, en:complete, en:nutrition-f...  Carrefour City         
2  [en:to-be-checked, en:complete, en:nutrition-f...       Carrefour         
3  [en:to-be-completed, e

In [57]:
df4["first_level_category"] = df4['categories_hierarchy'].map(lambda x: x[0].replace('en:', '').replace('fr:', ''))

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [63]:
df4["second_level_category"] = df4['categories_hierarchy'].map(lambda x: 
                                                               x[1].replace('en:', '').replace('fr:', '')
                                                              if len(x)>1 else None)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [65]:
df4["third_level_category"] = df4['categories_hierarchy'].map(lambda x: 
                                                               x[2].replace('en:', '').replace('fr:', '')
                                                              if len(x)>2 else None)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [66]:
df4.head()

_id                                          _keywords allergens  \
0       50097852  [anti-trace, hygiene, alcool, bille, motionsen...             
1  3560070272426  [deodorant, musk, stick, carrefour, hygiene, a...             
2  3560070937493  [lait, le, tolerance, sou, corp, dermatologiqu...             
3  3560071001308  [karite, beurre, et, de, musquee, huile, carre...             
4  3560070416974  [la, intime, carrefour, bio, gel, ecocert, pou...             

                    brands                 brands_tags  \
0          Rexona,Unilever          [rexona, unilever]   
1                Carrefour                 [carrefour]   
2                Carrefour                 [carrefour]   
3                Carrefour                 [carrefour]   
4  Carrefour Bio,Carrefour  [carrefour-bio, carrefour]   

                                          categories  \
0  Hygiène,Déodorants anti-transpirants,Déodorant...   
1               Hygiène,Déodorants,Déodorants sticks   
2                                Laits pour le corps   
3                                          Épilation   
4  Hygiène,Hygiène intime,Gels pour la toilette i...   

                                categories_hierarchy  \
0  [en:hygiene, en:anti-perspirants, en:deodorant...   
1  [en:hygiene, en:deodorants, fr:deodorants-sticks]   
2                           [en:body, en:body-milks]   
3                      [en:hygiene, en:hair-removal]   
4  [en:hygiene, en:intimate-hygiene, fr:gels-pour...   

                                     categories_tags           code  \
0  [en:hygiene, en:anti-perspirants, en:deodorant...       50097852   
1  [en:hygiene, en:deodorants, fr:deodorants-sticks]  3560070272426   
2                           [en:body, en:body-milks]  3560070937493   
3                      [en:hygiene, en:hair-removal]  3560071001308   
4  [en:hygiene, en:intimate-hygiene, fr:gels-pour...  3560070416974   

                                          codes_tags  \
0  [code-8, 50097852, 5009785x, 500978xx, 50097xx...   
1  [code-13, 3560070272426, 356007027242x, 356007...   
2  [code-13, 3560070937493, 356007093749x, 356007...   
3  [code-13, 3560071001308, 356007100130x, 356007...   
4  [code-13, 3560070416974, 356007041697x, 356007...   

               ...              serving_size                          sortkey  \
0              ...                                                 1487927561   
1              ...                                                 1491076400   
2              ...                            {u'$numberLong': u'1459971277'}   
3              ...                                                  487925728   
4              ...                            {u'$numberLong': u'1463774184'}   

                                              states  \
0  en:to-be-checked, en:complete, en:nutrition-fa...   
1  en:to-be-checked, en:complete, en:nutrition-fa...   
2  en:to-be-checked, en:complete, en:nutrition-fa...   
3  en:to-be-completed, en:nutrition-facts-complet...   
4  en:to-be-checked, en:complete, en:nutrition-fa...   

                                    states_hierarchy  \
0  [en:to-be-checked, en:complete, en:nutrition-f...   
1  [en:to-be-checked, en:complete, en:nutrition-f...   
2  [en:to-be-checked, en:complete, en:nutrition-f...   
3  [en:to-be-completed, en:nutrition-facts-comple...   
4  [en:to-be-checked, en:complete, en:nutrition-f...   

                                         states_tags          stores traces  \
0  [en:to-be-checked, en:complete, en:nutrition-f...        Franprix          
1  [en:to-be-checked, en:complete, en:nutrition-f...  Carrefour City          
2  [en:to-be-checked, en:complete, en:nutrition-f...       Carrefour          
3  [en:to-be-completed, en:nutrition-facts-comple...       Carrefour          
4  [en:to-be-checked, en:complete, en:nutrition-f...       Carrefour          

  first_level_category second_level_category          third_level_category  
0              hygiene    

In [51]:
'candy'.strip('can')

'dy'

In [68]:
writer = pd.ExcelWriter('cleaned_data.xlsx')
df4.to_excel(writer,'Sheet1')

writer.save()

In [41]:
df4.to_json('cleaned_data1.json', orient='records')

In [42]:
### check to make sure the data saved as intended
cleaned_data = pd.read_json('cleaned_data1.json')

In [43]:
cleaned_data.head()

_id                                          _keywords allergens  \
0       50097852  [anti-trace, hygiene, alcool, bille, motionsen...             
1  3560070272426  [deodorant, musk, stick, carrefour, hygiene, a...             
2  3560070937493  [lait, le, tolerance, sou, corp, dermatologiqu...             
3  3560071001308  [karite, beurre, et, de, musquee, huile, carre...             
4  3560070416974  [la, intime, carrefour, bio, gel, ecocert, pou...             

                    brands                 brands_tags  \
0          Rexona,Unilever          [rexona, unilever]   
1                Carrefour                 [carrefour]   
2                Carrefour                 [carrefour]   
3                Carrefour                 [carrefour]   
4  Carrefour Bio,Carrefour  [carrefour-bio, carrefour]   

                                          categories  \
0  Hygiène,Déodorants anti-transpirants,Déodorant...   
1               Hygiène,Déodorants,Déodorants sticks   
2                                Laits pour le corps   
3                                          Épilation   
4  Hygiène,Hygiène intime,Gels pour la toilette i...   

                                categories_hierarchy  \
0  [en:hygiene, en:anti-perspirants, en:deodorant...   
1  [en:hygiene, en:deodorants, fr:deodorants-sticks]   
2                           [en:body, en:body-milks]   
3                      [en:hygiene, en:hair-removal]   
4  [en:hygiene, en:intimate-hygiene, fr:gels-pour...   

                                     categories_tags           code  \
0  [en:hygiene, en:anti-perspirants, en:deodorant...       50097852   
1  [en:hygiene, en:deodorants, fr:deodorants-sticks]  3560070272426   
2                           [en:body, en:body-milks]  3560070937493   
3                      [en:hygiene, en:hair-removal]  3560071001308   
4  [en:hygiene, en:intimate-hygiene, fr:gels-pour...  3560070416974   

                                          codes_tags  ...   quantity  \
0  [code-8, 50097852, 5009785x, 500978xx, 50097xx...  ...      50 ml   
1  [code-13, 3560070272426, 356007027242x, 356007...  ...      50 ml   
2  [code-13, 3560070937493, 356007093749x, 356007...  ...     400 ml   
3  [code-13, 3560071001308, 356007100130x, 356007...  ...     100 ml   
4  [code-13, 3560070416974, 356007041697x, 356007...  ...     200 ml   

                       rev        serving_quantity serving_size  \
0                       14                       0                
1                       13                       0                
2  {u'$numberLong': u'11'}  {u'$numberLong': u'0'}                
3                        8                       0                
4  {u'$numberLong': u'15'}  {u'$numberLong': u'0'}                

                           sortkey  \
0                       1487927561   
1                       1491076400   
2  {u'$numberLong': u'1459971277'}   
3                        487925728   
4  {u'$numberLong': u'1463774184'}   

                                              states  \
0  en:to-be-checked, en:complete, en:nutrition-fa...   
1  en:to-be-checked, en:complete, en:nutrition-fa...   
2  en:to-be-checked, en:complete, en:nutrition-fa...   
3  en:to-be-completed, en:nutrition-facts-complet...   
4  en:to-be-checked, en:complete, en:nutrition-fa...   

                                    states_hierarchy  \
0  [en:to-be-checked, en:complete, en:nutrition-f...   
1  [en:to-be-checked, en:complete, en:nutrition-f...   
2  [en:to-be-checked, en:complete, en:nutrition-f...   
3  [en:to-be-completed, en:nutrition-facts-comple...   
4  [en:to-be-checked, en:complete, en:nutrition-f...   

                                         states_tags          stores traces  
0  [en:to-be-checked, en:complete, en:nutrition-f...        Franprix         
1  [en:to-be-checked, en:complete, en:nutrition-f...  Carrefour City         
2  [en:to-be-checked, en:complete, en:nutrition-f...       Carrefour         
3  [en:to-be-completed, e

In [44]:
cleaned_data.shape

(2293, 60)